In [ ]:
import pickle 

import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf

import os
os.chdir('..')

from performance_forecasting.data_creator import download_statement_data_for_exchanges

In [ ]:
def get_predictions(
    input_data, 
    sess, 
    batch_size,
    placeholder_varname,
    prediction_op, 
    dtype=np.float32
):
    input_data = input_data.reset_index(drop=True)
    preds = pd.DataFrame(
        columns=['next_year_revenue_pred', 'next_year_freeCashFlow_pred'],
        data=np.zeros([len(input_data), 2])
        
    )
    _input_cols = [col for col in input_data.columns if col != 'symbol']

    for j in range(0, len(input_data), batch_size): 
        preds.iloc[j: j+batch_size] = sess.run(
            prediction_op,
            {
                placeholder_varname: input_data.iloc[
                    j: j+batch_size
                ][_input_cols]
            }
        )
        
    preds['symbol'] = input_data['symbol']
    
    return preds
            

def plot_predictions(
    input_data, 
    targets, 
    sess, 
    batch_size, 
    placeholder_varname, 
    prediction_op, 
    dtype=np.float32
):
 
    preds = get_predictions(
        input_data, sess, batch_size, placeholder_varname, prediction_op, dtype
    )
    
    results = pd.concat([targets, preds], axis=1)
    
    plt = px.scatter(
        results,
        x='next_year_revenue_bn_scale_tg',
        y='next_year_revenue_pred'
    )
    plt.show()
    
    plt = px.scatter(
        results,
        x='next_year_freeCashFlow_bn_scale_tg',
        y='next_year_freeCashFlow_pred'
    )
    plt.show()
    return preds

In [ ]:
# allow_pickle=True    
        
class Dataset():
    def __init__(self, dataset_affix):
        dataset_names = ['train', 'test', 'val', 'next_year']
        dataset = {}

        for key in dataset_names:
            dataset['x_'+key] = pd.read_csv(
                f'data/performance_forecasting/{key}_input{dataset_affix}.csv', 
                index_col=0
            )
            dataset['y_'+key] = pd.read_csv(
                f'data/performance_forecasting/{key}_output{dataset_affix}.csv', 
                index_col=0
            )

        self.x_train, self.y_train = dataset['x_train'], dataset['y_train']
        self.x_test, self.y_test = dataset['x_test'], dataset['y_test']
        self.x_val, self.y_val = dataset['x_val'], dataset['y_val']
        self.x_val, self.y_val = dataset['x_val'], dataset['y_val']
        self.x_next_year, self.y_next_year = dataset['x_next_year'], dataset['y_next_year']

In [ ]:
[
    p.strip('train_input').strip('.npy') 
    for p in os.listdir('data/performance_forecasting/') 
    if 'train_input' in p
] 

In [ ]:
run = 1

dir_path = 'performance_forecasting/models/'+str(run)+'/'

params = pickle.load(open(dir_path+'model.pkl', 'rb'), errors='latin1')
dataset = Dataset(params['dataset_affix'])


df = None
next_year_preds = None

with tf.compat.v1.Session(graph=tf.Graph()) as sess:
    tf.compat.v1.saved_model.loader.load(sess, ["serve"], dir_path+'tf')
    graph = tf.compat.v1.get_default_graph()

    plot_predictions(
        dataset.x_test, 
        dataset.y_test,
        sess, 
        batch_size=32,
        placeholder_varname='x_placeholder_saved:0',
        prediction_op='prediction:0', 
    )
    
    next_year_preds = get_predictions(
        dataset.x_next_year, 
        sess, 
        batch_size=32,
        placeholder_varname='x_placeholder_saved:0',
        prediction_op='prediction:0', 
    )
    


In [ ]:
dataset.y_val

In [ ]:
dataset.y_val.loc[
    dataset.y_val['symbol'] == 'SUMO', 
    ['next_year_revenue_bn_scale_tg', 'next_year_freeCashFlow_bn_scale_tg']
]*1000000

In [ ]:
next_year_preds.loc[
    next_year_preds['symbol'] == 'SUMO', 
    ['next_year_revenue_pred', 'next_year_freeCashFlow_pred']
]*1000000

In [ ]:
next_year_preds['symbol'].value_counts()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="free_cashlow_pred", y="free_cashlow")
fig.show()

In [ ]:
"""
    for var in tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.VARIABLES)
    for v in tf.compat.v1.get_default_graph().get_operations()
    for n in tf.compat.v1.get_default_graph().as_graph_def().node
    all_vars = tf.global_variables()
    tf.compat.v1.get_default_graph().get_tensor_by_name('x_placeholder')
    tf.compat.v1.get_default_graph().get_operation_by_name('x_placeholder')
"""